In [1]:
import os
import shutil

def copy_and_rename_dwi_files(src_base, dest_base, count=20):
    """
    复制 src_base 目录下前 count 个子文件夹中的 DWI 文件，
    并按要求重命名后保存到 dest_base/1,2,...,count 中。
    """
    # 获取按字母序排序的子文件夹列表
    all_folders = sorted([
        f for f in os.listdir(src_base)
        if os.path.isdir(os.path.join(src_base, f))
    ])
    selected_folders = all_folders[:count]

    # 源文件名到目标文件名的映射
    file_mapping = {
        "DWI.nii.gz": "data.nii.gz",
        "DWI.bvec": "bvecs",
        "DWI.bval": "bvals",
        "b0_brain_mask.nii.gz": "nodif_brain_mask.nii.gz"
    }

    for idx, folder in enumerate(selected_folders, start=1):
        src_folder = os.path.join(src_base, folder)
        dst_folder = os.path.join(dest_base, str(idx))
        os.makedirs(dst_folder, exist_ok=True)

        for src_name, dst_name in file_mapping.items():
            src_file = os.path.join(src_folder, src_name)
            dst_file = os.path.join(dst_folder, dst_name)
            if os.path.exists(src_file):
                shutil.copy2(src_file, dst_file)
                print(f"Copied: {src_file} -> {dst_file}")
            else:
                print(f"Warning: 未找到 {src_file}，已跳过。")

    print("全部复制完成！")

if __name__ == "__main__":
    src_base = r"C:\Users\myp\Desktop\NC-"
    dest_base = r"E:\gulou"
    copy_and_rename_dwi_files(src_base, dest_base, count=20)


Copied: C:\Users\myp\Desktop\NC-\AN_DIAN_CHENG_1806280907_570108948\DWI.nii.gz -> E:\gulou\1\data.nii.gz
Copied: C:\Users\myp\Desktop\NC-\AN_DIAN_CHENG_1806280907_570108948\DWI.bvec -> E:\gulou\1\bvecs
Copied: C:\Users\myp\Desktop\NC-\AN_DIAN_CHENG_1806280907_570108948\DWI.bval -> E:\gulou\1\bvals
Copied: C:\Users\myp\Desktop\NC-\AN_DIAN_CHENG_1806280907_570108948\b0_brain_mask.nii.gz -> E:\gulou\1\nodif_brain_mask.nii.gz
Copied: C:\Users\myp\Desktop\NC-\CHANG_YUE_PING_2209232807_695887160\DWI.nii.gz -> E:\gulou\2\data.nii.gz
Copied: C:\Users\myp\Desktop\NC-\CHANG_YUE_PING_2209232807_695887160\DWI.bvec -> E:\gulou\2\bvecs
Copied: C:\Users\myp\Desktop\NC-\CHANG_YUE_PING_2209232807_695887160\DWI.bval -> E:\gulou\2\bvals
Copied: C:\Users\myp\Desktop\NC-\CHANG_YUE_PING_2209232807_695887160\b0_brain_mask.nii.gz -> E:\gulou\2\nodif_brain_mask.nii.gz
Copied: C:\Users\myp\Desktop\NC-\CHEN_LU_2306302660_725417238\DWI.nii.gz -> E:\gulou\3\data.nii.gz
Copied: C:\Users\myp\Desktop\NC-\CHEN_LU_2306

In [1]:
#!/usr/bin/env python3
import os
import shutil
import numpy as np
import nibabel as nib
import sys

# -------- 配置区 --------
src_base = r"C:\Users\myp\Desktop\MSA-Ma"    # 源数据根目录
dest_base = r"C:\Users\myp\Desktop\MSA-Ma"        # 目标工作目录
max_subjects = 20                            # 最多处理前 20 个被试
# ------------------------

def find_b0_index(bval_path):
    """读取 bval 文件，返回第一个等于 0 的索引（0-based）。"""
    bvals = np.loadtxt(bval_path)
    zeros = np.where(bvals == 0)[0]
    if zeros.size == 0:
        raise ValueError(f"No b0 volume found in {bval_path}")
    return int(zeros[0])

def extract_b0(data_path, b0_idx, out_path):
    """从 data.nii.gz 中提取第 b0_idx 个体积，保存为 out_path。"""
    img = nib.load(data_path)
    data = img.get_fdata()
    # 提取第 b0_idx 个 4D 体积
    b0_vol = data[..., b0_idx]
    b0_img = nib.Nifti1Image(b0_vol, img.affine, img.header)
    nib.save(b0_img, out_path)

def main():
    # 检查路径
    if not os.path.isdir(src_base):
        print(f"Error: 源目录不存在: {src_base}", file=sys.stderr)
        sys.exit(1)
    os.makedirs(dest_base, exist_ok=True)

    # 列出所有 HC_sub* 文件夹，按名称排序
    subs = sorted([d for d in os.listdir(src_base)
                   if os.path.isdir(os.path.join(src_base, d))
                   and d.startswith("HC_sub")])
    for idx, sub in enumerate(subs[:max_subjects], start=1):
        src_subdir = os.path.join(src_base, sub)
        dst_subdir = os.path.join(dest_base, f"sub{idx}")
        os.makedirs(dst_subdir, exist_ok=True)

        # 1. 复制并重命名 T1
        t1_src = os.path.join(src_subdir, "T1", "T1_reoriented.nii.gz")
        t1_dst = os.path.join(dst_subdir, f"T1_sub{idx}.nii.gz")
        if os.path.isfile(t1_src):
            shutil.copy2(t1_src, t1_dst)
            print(f"[{idx:02d}] Copied T1 → {t1_dst}")
        else:
            print(f"[{idx:02d}] Warning: 找不到 {t1_src}")

        # 2. 提取并保存 b0
        bval_file = os.path.join(src_subdir, "dti.bval")
        data_file = os.path.join(src_subdir, "data.nii.gz")
        try:
            b0_idx = find_b0_index(bval_file)
            b0_dst = os.path.join(dst_subdir, f"b0_sub{idx}.nii.gz")
            extract_b0(data_file, b0_idx, b0_dst)
            print(f"[{idx:02d}] Extracted b0 (vol #{b0_idx}) → {b0_dst}")
        except Exception as e:
            print(f"[{idx:02d}] Error extracting b0: {e}", file=sys.stderr)

    print("\n整理完成！所有被试数据已放入：", dest_base)

if __name__ == "__main__":
    main()


e:\anaconda3\envs\env_torch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
e:\anaconda3\envs\env_torch\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
e:\anaconda3\envs\env_torch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


[01] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub1\T1_sub1.nii.gz
[01] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub1\b0_sub1.nii.gz
[02] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub2\T1_sub2.nii.gz
[02] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub2\b0_sub2.nii.gz
[03] Warning: 找不到 C:\Users\myp\Desktop\MSA-Ma\HC_sub03\T1\T1_reoriented.nii.gz
[03] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub3\b0_sub3.nii.gz
[04] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub4\T1_sub4.nii.gz
[04] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub4\b0_sub4.nii.gz
[05] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub5\T1_sub5.nii.gz
[05] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub5\b0_sub5.nii.gz
[06] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub6\T1_sub6.nii.gz
[06] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub6\b0_sub6.nii.gz
[07] Copied T1 → C:\Users\myp\Desktop\MSA-Ma\sub7\T1_sub7.nii.gz
[07] Extracted b0 (vol #0) → C:\Users\myp\Desktop\MSA-Ma\sub7\b0_sub7

In [3]:
#!/usr/bin/env python3
import os
import shutil
import sys

# -------- 配置区 --------
src_base = r"C:\Users\myp\Desktop\MSA-Ma"         # 原始 20 个 HC_sub01…HC_sub20 文件夹所在目录
dest_base = r"C:\Users\myp\Desktop\bedpostx_input"  # 目标目录，用于存放 bedpostx 输入
max_subjects = 20                                 # 最多处理前 20 个文件夹
# ------------------------

def collect_for_bedpostx(src_base, dest_base, max_subs=20):
    if not os.path.isdir(src_base):
        print(f"Error: 源目录不存在: {src_base}", file=sys.stderr)
        sys.exit(1)
    os.makedirs(dest_base, exist_ok=True)

    # 找到 HC_sub01…HC_sub20
    subs = sorted([
        d for d in os.listdir(src_base)
        if os.path.isdir(os.path.join(src_base, d)) and d.startswith("HC_sub")
    ])[:max_subs]

    # 要复制的文件列表，新增 nodif_brain_mask.nii.gz
    files_to_copy = [
        "data.nii.gz",
        "dti.bvec",
        "dti.bval",
        "nodif_brain_mask.nii.gz"
    ]

    for sub in subs:
        src_subdir = os.path.join(src_base, sub)
        dst_subdir = os.path.join(dest_base, sub)
        os.makedirs(dst_subdir, exist_ok=True)

        for fname in files_to_copy:
            src_file = os.path.join(src_subdir, fname)
            dst_file = os.path.join(dst_subdir, fname)
            if os.path.isfile(src_file):
                shutil.copy2(src_file, dst_file)
                print(f"[{sub}] Copied {fname}")
            else:
                print(f"[{sub}] Warning: {fname} not found, skipped.")

    print(f"\nDone! All data collected under:\n  {dest_base}")

if __name__ == "__main__":
    collect_for_bedpostx(src_base, dest_base, max_subjects)


[HC_sub01] Copied data.nii.gz
[HC_sub01] Copied dti.bvec
[HC_sub01] Copied dti.bval
[HC_sub01] Copied nodif_brain_mask.nii.gz
[HC_sub02] Copied data.nii.gz
[HC_sub02] Copied dti.bvec
[HC_sub02] Copied dti.bval
[HC_sub02] Copied nodif_brain_mask.nii.gz
[HC_sub03] Copied data.nii.gz
[HC_sub03] Copied dti.bvec
[HC_sub03] Copied dti.bval
[HC_sub03] Copied nodif_brain_mask.nii.gz
[HC_sub04] Copied data.nii.gz
[HC_sub04] Copied dti.bvec
[HC_sub04] Copied dti.bval
[HC_sub04] Copied nodif_brain_mask.nii.gz
[HC_sub05] Copied data.nii.gz
[HC_sub05] Copied dti.bvec
[HC_sub05] Copied dti.bval
[HC_sub05] Copied nodif_brain_mask.nii.gz
[HC_sub06] Copied data.nii.gz
[HC_sub06] Copied dti.bvec
[HC_sub06] Copied dti.bval
[HC_sub06] Copied nodif_brain_mask.nii.gz
[HC_sub07] Copied data.nii.gz
[HC_sub07] Copied dti.bvec
[HC_sub07] Copied dti.bval
[HC_sub07] Copied nodif_brain_mask.nii.gz
[HC_sub08] Copied data.nii.gz
[HC_sub08] Copied dti.bvec
[HC_sub08] Copied dti.bval
[HC_sub08] Copied nodif_brain_mask

In [1]:
#!/usr/bin/env python3
import os
import gzip
import shutil
import sys

def decompress_nii_gz(root_dir):
    """
    递归遍历 root_dir，解压所有 .nii.gz 文件为 .nii，并删除原 .nii.gz。
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.lower().endswith('.nii.gz'):
                gz_path = os.path.join(dirpath, fname)
                nii_path = gz_path[:-3]  # 去掉末尾的 .gz

                try:
                    # 解压
                    with gzip.open(gz_path, 'rb') as f_in, open(nii_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    # 删除原 .nii.gz
                    os.remove(gz_path)
                    print(f"[OK] 解压并删除: {gz_path}")
                except Exception as e:
                    print(f"[ERROR] 处理 {gz_path} 时出错: {e}", file=sys.stderr)

if __name__ == '__main__':
    # 如果要从命令行指定目录，可取消下面两行注释：
    # if len(sys.argv) != 2:
    #     print(f"用法: {sys.argv[0]} <WORKING_DIR 路径>", file=sys.stderr); sys.exit(1)
    # root = sys.argv[1]

    root = r"C:\Users\myp\Desktop\WORKING_DIR"  # <-- 修改为你的目录
    if not os.path.isdir(root):
        print(f"Error: 指定的目录不存在: {root}", file=sys.stderr)
        sys.exit(1)

    decompress_nii_gz(root)
    print("所有 .nii.gz 已处理完毕。")


[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\ACR_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\ACR_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\ALIC_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\ALIC_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\BCC.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CGC_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CGC_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CGH_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CGH_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CP_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CP_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CST_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\CST_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\EC_L.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\EC_R.nii.gz
[OK] 解压并删除: C:\Users\myp\Desktop\WORKING_DIR\ROI\Fx.nii.gz
[OK] 解压并删除: C:\

In [2]:
#!/usr/bin/env python3
import os
import shutil
import sys

# -------- 配置区 --------
hc_base    = r"C:\Users\myp\Desktop\HC"               # HC_sub01–HC_sub19 所在目录
working_dir = r"C:\Users\myp\Desktop\WORKING_DIR"     # 目标工作目录，包含 sub1–sub19
n_subjects = 19                                       # 子文件夹数量
# ------------------------

def copy_and_rename_bedpostx(hc_base, working_dir, n_subjects):
    if not os.path.isdir(hc_base):
        print(f"Error: HC 根目录不存在: {hc_base}", file=sys.stderr)
        sys.exit(1)
    os.makedirs(working_dir, exist_ok=True)

    for i in range(1, n_subjects+1):
        # 构造源路径：HC_sub01 … HC_sub19
        hc_sub_name = f"HC_sub{i:02d}"
        src_subdir  = os.path.join(hc_base, hc_sub_name)
        src_bedpost = os.path.join(src_subdir, "data.bedpostX")

        # 构造目标路径：WORKING_DIR/sub1 … sub19
        dst_subdir  = os.path.join(working_dir, f"sub{i}")
        dst_bedpost = os.path.join(dst_subdir, "DTI.bedpostX")

        # 检查源
        if not os.path.isdir(src_bedpost):
            print(f"[{hc_sub_name}] Warning: 找不到 {src_bedpost}，已跳过。")
            continue

        # 确保目标子文件夹存在
        os.makedirs(dst_subdir, exist_ok=True)

        # 如果目标已存在，则先删除以便重新复制
        if os.path.exists(dst_bedpost):
            shutil.rmtree(dst_bedpost)

        # 复制并重命名
        try:
            shutil.copytree(src_bedpost, dst_bedpost)
            print(f"[{hc_sub_name}] 成功复制到 {dst_bedpost}")
        except Exception as e:
            print(f"[{hc_sub_name}] 复制失败: {e}", file=sys.stderr)

    print("\n全部处理完毕。")

if __name__ == "__main__":
    copy_and_rename_bedpostx(hc_base, working_dir, n_subjects)


[HC_sub01] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub1\DTI.bedpostX
[HC_sub02] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub2\DTI.bedpostX
[HC_sub03] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub3\DTI.bedpostX
[HC_sub04] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub4\DTI.bedpostX
[HC_sub05] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub5\DTI.bedpostX
[HC_sub06] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub6\DTI.bedpostX
[HC_sub07] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub7\DTI.bedpostX
[HC_sub08] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub8\DTI.bedpostX
[HC_sub09] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub9\DTI.bedpostX
[HC_sub10] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub10\DTI.bedpostX
[HC_sub11] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub11\DTI.bedpostX
[HC_sub12] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub12\DTI.bedpostX
[HC_sub13] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub13\DTI.bedpostX
[HC_sub14] 成功复制到 C:\Users\myp\Desktop\WORKING_DIR\sub14\DTI.bedpostX
[HC_sub15] 成功复制到 C:\Users\myp\Desktop\WORKI

In [3]:
#!/usr/bin/env python3
import os
import sys

# -------- 配置区 --------
working_dir = r"C:\Users\myp\Desktop\WORKING_DIR"  # 修改为你的 WORKING_DIR 路径
output_fname = "subjects.txt"                     # 输出的 txt 文件名
n_subjects = 19                                   # 要写入的子文件夹数量
# ------------------------

def write_subject_list(working_dir, output_fname, n_subjects):
    if not os.path.isdir(working_dir):
        print(f"Error: 目录不存在: {working_dir}", file=sys.stderr)
        sys.exit(1)

    out_path = os.path.join(working_dir, output_fname)
    with open(out_path, 'w', encoding='utf-8') as f:
        for i in range(1, n_subjects + 1):
            f.write(f"sub{i}\n")
    print(f"已生成：{out_path}")

if __name__ == "__main__":
    write_subject_list(working_dir, output_fname, n_subjects)


已生成：C:\Users\myp\Desktop\WORKING_DIR\subjects.txt


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import shutil
from pathlib import Path

def copy_xuanwu(source_root: Path, dest_root: Path):
    # 确保目标根目录存在
    dest_root.mkdir(parents=True, exist_ok=True)

    # 遍历所有以 HC_sub 开头的目录
    for sub in source_root.iterdir():
        if sub.is_dir() and sub.name.startswith("HC_sub"):
            dst_sub = dest_root / sub.name
            dst_sub.mkdir(exist_ok=True)

            # 需要复制的文件列表
            filenames = [
                "data.nii.gz",
                "dti.bval",
                "dti.bvec",
                "nodif_brain.nii.gz",
                "nodif_brain_mask.nii.gz"
            ]

            for name in filenames:
                src_file = sub / name
                if src_file.exists():
                    shutil.copy2(src_file, dst_sub / name)
                else:
                    print(f"[WARN] 未找到文件: {src_file}")

            # 处理 T1 重命名
            src_t1 = sub / "T1" / "T1_reoriented.nii.gz"
            if src_t1.exists():
                dst_t1 = dst_sub / "T1.nii.gz"
                shutil.copy2(src_t1, dst_t1)
            else:
                print(f"[WARN] 未找到 T1 文件: {src_t1}")

    print("复制完成。")

if __name__ == "__main__":
    # 根据需要修改源和目标路径
    source_root = Path(r"C:\Users\myp\Desktop\Xuanwu")
    dest_root   = Path(r"C:\Users\myp\Desktop\Xuanwu_Copied")

    copy_xuanwu(source_root, dest_root)


复制完成。
